In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Configuración de estilo (opcional, pero mejora la estética)
sns.set_style('whitegrid') 
plt.rcParams['figure.figsize'] = (10, 6)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Asegúrate de que tu archivo de datos se llame 'retailmax.csv' y esté en la carpeta 'data'
df = pd.read_csv('./data/retailmax.csv') 

# Es importante renombrar las columnas si el CSV usa el nombre 'Mall_Customers.csv'
# Si tu archivo 'retailmax.csv' ya tiene las columnas renombradas, puedes omitir esto
if 'Annual Income (k$)' not in df.columns:
    df.columns = ['CustomerID', 'Gender', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']

print("Datos cargados. Vista previa:")
print(df.head())

In [ ]:
# Seleccionar las variables numéricas clave para el clustering
features = df[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']]

print("Features seleccionadas (Age, Income, Score):")
print(features.head())

In [ ]:
# Inicializar y ajustar el escalador
scaler = StandardScaler()

# Estandarizar las features (media=0, desviación estándar=1)
scaled_features = scaler.fit_transform(features)

print("Datos estandarizados creados.")
print("Primeros 3 registros estandarizados:")
print(scaled_features[0:3])

In [ ]:
# Verificar el tipo del objeto resultante (debe ser un array de NumPy)
type(scaled_features)

In [ ]:
# Implementación inicial de K-Means con K=3
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

print("K-Means con 3 clusters ejecutado.")
print(kmeans)

In [ ]:
# Mostrar las asignaciones de clusters resultantes
clusters

In [ ]:
# Verificar que el número de etiquetas de cluster sea igual al número de filas
len(clusters)

In [ ]:
# Prueba con K=5 (El que se sugiere es óptimo en el método del codo)
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

print("K-Means con 5 clusters ejecutado.")
print(kmeans)

In [ ]:
# Mostrar las asignaciones de clusters resultantes para K=5
clusters

In [ ]:
# Prueba con K=4 (Solo para demostración, ya que K=5 parece mejor)
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

print("K-Means con 4 clusters ejecutado.")

In [ ]:
# Mostrar las asignaciones de clusters resultantes para K=4
clusters

In [ ]:
# Mostrar el DataFrame sin la columna de clasificación aún
df

In [ ]:
# Asignar la última asignación de clusters (K=4) al DataFrame original
# NOTA: Si queremos usar K=5 (el óptimo), debemos re-ejecutar la Celda 9 antes.
# Usaré la asignación de K=4 aquí siguiendo tu secuencia, pero lo óptimo es K=5.
df['clasificación'] = clusters
df

In [ ]:
# Contar cuántos clientes hay en cada cluster
df['clasificación'].value_counts()

In [ ]:
# Graficar los clusters en el plano Ingreso vs. Gasto (las dimensiones más discriminatorias)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, 
                x='Annual Income (k$)', 
                y='Spending Score (1-100)', 
                hue='clasificación', 
                palette='Set1', 
                s=80)
plt.title('Clusters de Clientes (K=4)', fontsize=16)
plt.xlabel('Ingreso Anual (k$)')
plt.ylabel('Puntuación de Gasto (1-100)')
plt.legend(title='Cluster')
plt.show()

In [ ]:
# Función para calcular la Suma de Cuadrados Dentro del Cluster (WCSS)
def calcular_wcss(datos):
    wcss = []
    for n in range(1, 11):
        # Ajustar K-Means para cada K
        kmeans = KMeans(n_clusters=n, init='k-means++', max_iter=300, n_init=10, random_state=42)
        kmeans.fit(datos)
        # inertia_ es el WCSS
        wcss.append(kmeans.inertia_)
    return wcss

# Calcular el WCSS para diferentes números de clusters
wcss = calcular_wcss(scaled_features)

# Graficar el método del codo
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--', color='blue')
plt.title('Método del Codo (WCSS vs. Número de Clusters)', fontsize=16)
plt.xlabel('Número de clusters')
plt.ylabel('WCSS (Inercia)')
plt.xticks(range(1, 11))
plt.grid(True)
plt.show()

print("La gráfica muestra que el 'codo' está en K=4 o K=5, siendo K=5 el punto donde la ganancia marginal comienza a disminuir significativamente.")